In [26]:
import pandas as pd
import numpy as np
df=pd.read_csv(r"C:\Users\Akhil M\Downloads\sentiment analysis.csv",encoding="ISO-8859-1")
df.columns=['target','id','data','flag','name','text']


In [27]:
df.shape

(1599999, 6)

In [5]:
df=df.drop(['id','data','flag','name'],axis=1)
df.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [8]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # remove URLs
    text = re.sub(r"@\w+", "", text)  # remove mentions
    text = re.sub(r"#\w+", "", text)  # remove hashtags
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\d+", "", text)  # remove numbers
    text = re.sub(r"\s+", " ", text).strip()  # remove extra spaces
    return text

In [11]:
df["clean_text"] = df["text"].astype(str).apply(clean_text)
df["label"] = df["target"].map({0: "Negative", 4: "Positive"})
df=df.drop(['target','text'],axis=1)

In [12]:
df.head()

,clean_text,label
0,is upset that he cant update his facebook by t...,Negative
1,i dived many times for the ball managed to sav...,Negative
2,my whole body feels itchy and like its on fire,Negative
3,no its not behaving at all im mad why am i her...,Negative
4,not the whole crew,Negative


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"], df["label"], test_size=0.2, random_state=42
)

In [20]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [21]:
model=LogisticRegression()
model.fit(X_train_tfidf,y_train)

LogisticRegression()

In [23]:
pred=model.predict(X_test_tfidf)
print(f'accuracy score is : {accuracy_score(y_test,pred)}')

accuracy score is : 0.790390625


In [24]:
print("Classification Report:\n", classification_report(y_test, pred))


Classification Report:
               precision    recall  f1-score   support

    Negative       0.80      0.78      0.79    159494
    Positive       0.78      0.80      0.79    160506

    accuracy                           0.79    320000
   macro avg       0.79      0.79      0.79    320000
weighted avg       0.79      0.79      0.79    320000



In [25]:
import joblib
import os

# Create directory to store the model
os.makedirs("models", exist_ok=True)

# Save model and vectorizer
joblib.dump(model, "models/sentiment_model.pkl")
joblib.dump(tfidf, "models/tfidf_vectorizer.pkl")

print("✅ Model and Vectorizer saved successfully!")


✅ Model and Vectorizer saved successfully!
